In [ ]:
from IPython.display import HTML
HTML(open('../style.css').read())

## Drawing Abstract Syntax Trees with GraphViz

In [ ]:
import graphviz as gv

In [ ]:
type AST = str | int | float | tuple[str, *tuple["AST", ...]]

`NodeMap`: A dictionary mapping a node's address (tuple of ints) to its name (string).

In [ ]:
type NodeMap = dict[tuple[int, ...], str]

The function `tuple2dot` takes a nested tuple `t` as its argument.  This nested tuple is interpreted as an
*abstract syntax tree*.   This tree is visualized using `graphviz`.

In [ ]:
def tuple2dot(t: AST) -> gv.Digraph:
    dot = gv.Digraph('Abstract Syntax Tree')
    Nodes_2_Names = {}
    assign_numbers((), t, Nodes_2_Names)
    create_nodes(dot, (), t, Nodes_2_Names)
    return dot

The function `assign_numbers` takes three arguments:
- `t` is a nested tuple that is interpreted as a tree,
- `Nodes2Numbers` is dictionary,
- `n` is a natural number.

Given a tree `t` that is represented as a nested tuple, the function `assign_numbers` assigns a unique natural number 
to every node of `t`.  This assignment is stored in the dictionary `Nodes2Numbers`. `n` is the first natural number
that is used.  The function returns the smallest natural number that is still unused.

In [ ]:
def assign_numbers(address: tuple[int, ...], 
                   t: AST, 
                   Nodes2Numbers: NodeMap, 
                   n: int =0) -> int:
    Nodes2Numbers[address] = str(n)
    if isinstance(t, str) or isinstance(t, int) or isinstance(t, float):
        return n + 1
    n += 1
    j  = 1
    for t in t[1:]:
        n = assign_numbers(address + (j,), t, Nodes2Numbers, n)
        j += 1
    return n

The function `create_nodes` takes three arguments:
- `dot` is an object of class `graphviz.digraph`,
- `t` is an abstract syntax tree represented as a nested tuple.
- `Nodes_2_Names`is a dictionary mapping nodes in `t` to unique names
   that can be used as node names in graphviz.
   
The function creates the nodes in `t` and connects them via directed edges so that `t` is represented as a tree.

In [ ]:
def create_nodes(dot: gv.Digraph, 
                 a: tuple[int, ...], 
                 t: AST, 
                 Nodes_2_Names: NodeMap) -> None:
    root = Nodes_2_Names[a]
    if isinstance(t, str) or isinstance(t, int) or isinstance(t, float):
        dot.node(root, label=str(t))
        return
    dot.node(root, label=t[0])
    j = 1
    for c in t[1:]:
        child = Nodes_2_Names[a + (j,)]
        dot.edge(root, child)
        create_nodes(dot, a + (j,), c, Nodes_2_Names)
        j += 1